## Statistical Correlation Overview

Statistical correlation is a measure that describes the extent to which two variables change together. In our context, we are interested in how the IMDb scores of 'Game of Thrones' episodes or seasons correlate with other variables (like episode or season number).

### Key Points:

- **Pearson Correlation Coefficient**: This is the most commonly used correlation measure. The Pearson correlation coefficient $ r $ is calculated as follows:

  $$
  r = \frac{\sum (X_i - \bar{X})(Y_i - \bar{Y})}{\sqrt{\sum (X_i - \bar{X})^2 \sum (Y_i - \bar{Y})^2}}
  $$

  Where:
  - $ X_i $ and $ Y_i $ are individual sample points.
  - $ \bar{X} $ and $ \bar{Y} $ are means of the samples $ X $ and $ Y $.
  - The summation is over all sample points.

  The coefficient ranges from -1 to 1, where 1 indicates a perfect positive correlation, -1 indicates a perfect negative correlation, and 0 indicates no correlation.

- **Positive vs Negative Correlation**:
  - *Positive Correlation*: As one variable increases, the other variable also increases.
  - *Negative Correlation*: As one variable increases, the other variable decreases.

- **Statistical Significance**: It’s important to determine whether a calculated correlation is statistically significant, which often requires further statistical testing.

### Usage in Our Analysis:

We will calculate the Pearson correlation coefficient to understand the relationship between IMDb scores and either episode numbers or season numbers. However, it's important to remember that correlation does not imply causation. A high or low correlation does not mean that one variable causes the other to change.
